# Imports

In [ ]:
%load_ext autoreload
%autoreload 2


import core.config_builders as cfgb

In [ ]:
# Initialize config.
config = cconfig.get_config_from_env()

# Execute

In [ ]:
if config is None:
    raise ValueError("No config provided.")

In [ ]:
if config["fail"]:
    raise ValueError("Fail.")
print("success")